In [1]:
import json

from datetime import datetime

import tkinter as tk
from tkinter.tix import Tk, Control, ComboBox, Button, Balloon, Label, Entry #升级的组合控件包

from tkinter.messagebox import showinfo, showwarning, showerror #各种类型的提示框
from tkinter.constants import *

In [41]:
root = Tk() # 初始化Tk()  root便是你布局的根节点了，以后的布局都在它之上

root.title("行者桑结") # 设置窗口标题

width = root.winfo_reqwidth() + 400 #窗口大小
height = root.winfo_reqheight() + 100 #窗口大小
x, y = (root.winfo_screenwidth()  - width )/2, (root.winfo_screenheight() - height)/2
root.geometry('%dx%d+%d+%d' % (width, height, x, y )) #窗口位置居中,设置窗口大小 注意：是x 不是*

root.resizable(width=True, height=True) # 设置窗口是否可以变化长/宽，False不可变，True可变，默认为True

root.tk.eval('package require Tix') #引入升级包，这样才能使用升级的组合控件

#创建一个状态显示标签
info_status = Label(root, width=300, relief=GROOVE, font=("Arial",13), fg='red', bd=1, anchor = SW)
info_status.place(x=0,y=0, relx=0/20,rely=7/8, relwidth=20/20,relheigh=3/30,anchor=NW)

#实现日期选择按钮，以及日期输入。通过气球和状态栏说明操作说明
date_button = Button(root, text = '转换日期:', relief=SUNKEN, font=("Arial",11),bd=2)
date_button.place(x=0,y=0, relx=2/20,rely=1/8, relwidth=3/20,relheigh=3/30)

date_str = tk.StringVar()
date = Entry(root, textvariable = date_str, bd=2).place(x=0,y=0, relx=5/20,rely=1/8, relwidth=6/20,relheigh=3/30)

date_balloon = Balloon(root, statusbar = info_status)
date_balloon.bind_widget(date_button, balloonmsg='', statusmsg='说明：点按这个按钮,选择牛熊状态改变的交易日期。')

#实现交易转换按钮
def bull_bear_button_call():
    global bull_label, bear_label
    if bull_button.configure('relief')[-1] == RAISED:
        bull_button.configure(state=DISABLED, relief=SUNKEN, bg='DarkRed')
        bull_label.configure(bg='DarkRed', fg='black')
        bear_button.configure(state=NORMAL, relief=RAISED, bg='green')
        bear_label.configure(bg='green', fg='white')
        
    else:
        bull_button.configure(state=NORMAL, relief=RAISED, bg='red')
        bull_label.config(bg='Red', fg='white')
        bear_button.configure(state=DISABLED, relief=SUNKEN, bg='DarkGreen')
        bear_label.configure(bg='DarkRed', fg='black')
        
bull_button = Button(root, text='BULL', bg="red", fg="white", command=bull_bear_button_call, state=NORMAL,
                     activebackground = "red", activeforeground = "black",
                     bd=5, font=("Arial",12), width=6, relief=RAISED)
bull_button.place(x=0,y=0, relx=2/20,rely=2.5/8, relwidth=2/20,relheigh=5/30, anchor=NW)

bear_button = Button(root, text='BEAR', bg='DarkGreen', fg="white",command=bull_bear_button_call, state=DISABLED,
                     activebackground = "green", activeforeground = "black",
                     bd=5, font=("Arial",12), width=6, relief=SUNKEN)
bear_button.place(x=0,y=0, relx=4/20,rely=2.5/8, relwidth=2/20,relheigh=5/30, anchor=NW)

bull_balloon = Balloon(root, statusbar=info_status)
bull_balloon.bind_widget(bull_button, balloonmsg='', statusmsg='说明：点按这个按钮,改变牛熊状态：红色代表牛，绿色代表熊。')

bear_balloon = Balloon(root, statusbar=info_status)
bear_balloon.bind_widget(bear_button, balloonmsg='', statusmsg='说明：点按这个按钮,改变牛熊状态：红色代表牛，绿色代表熊。')

#交易使用红色和绿色色条表示交易的进展
bull_label = Label(root, text = ' xxx Days', anchor=W, relief=SUNKEN, bg='red', fg="white", bd=0
                  )
bull_label.place(x=0,y=0, relx=2/20,rely=4/8, relwidth=6/20,relheigh=4/30)

bear_label = Label(root, text = ' xxx Days', anchor=W, relief=SUNKEN, bg='green', fg="black",bd=0
                  )
bear_label.place(x=0,y=0, relx=8/20,rely=4/8, relwidth=2/20,relheigh=4/30)

bull_bear_label = Label(root, text = ' xxx Days', anchor=CENTER,  relief=SUNKEN, bg='gray', bd=2
                ).place(x=0,y=0, relx=10/20,rely=4/8, relwidth=8/20,relheigh=4/30)

root.mainloop()

In [9]:
#如果没有days.json文件，则创建；如有，则读入
try:
    with open('C://Users//faqui//Documents//github//y_Python//bull_bear_days.json', 'r') as f:
        bull_bear_days = json.load(f)
except FileNotFoundError:
    bull_bear_days = {
        'bear_days': 0,
        'bull_days': 0,
        'bull_bear': [{'bull': '2020/01/01'}, 
                      ]
        }

    with open('C://Users//faqui//Documents//github//y_Python//bull_bear_days.json', 'w') as f:
        json.dump(bull_bear_days, f)
else:
    pass

In [ ]:
def trading_days(dt1, dt2):
    '''判断两个日期间的交易日数，含最后一个日期
    两个日期间的天数 - 周末 - 法定节假日'''
    
    holidays = ['2020/01/01', '2020/01/24', '2020/01/27', '2020/01/28', '2020/01/29', '2020/01/30', 
                '2020/04/06', '2020/05/01', '2020/05/04', '2020/05/05', '2020/06/25', '2020/06/26',
                '2020/10/01', '2020/10/02', '2020/10/05', '2020/10/06', '2020/10/07', '2020/10/08']
    
    #取出两个日期的数字范围，看假期是否在这个范围内，如在，则假期计数+1
    dt1num = int(''.join(dt1.split('/')))
    dt2num = int(''.join(dt2.split('/')))
    
    holiday_counts = 0
    
    for holiday in holidays:
        holiday_num = int(''.join(holiday.split('/')))
        if dt1num <= holiday_num and holiday_num <= dt2num :
            holiday_counts +=1
    
    #计算两个日期间的工作天数
    date1 = datetime.strptime(dt1,'%Y/%m/%d') #字符串日期转换datetiem格式
    date2 = datetime.strptime(dt2,'%Y/%m/%d')
    
    days1 = int(date1.strftime('%j')) 
    weekend_days1 = (days1 + 2) //7 *2 
    if date1.strftime('%a') == 'Sat' :  weekend_days1 += 1
    
    days2 = int(date2.strftime('%j'))
    weekend_days2 = (days2 + 2) //7 *2 
    if date2.strftime('%a') == 'Sat' :  weekend_days2 += 1
    
    #计算两个日期间的交易天数
    trading_days = (days2 - weekend_days2) - (days1 - weekend_days1) - holiday_counts
    
    return trading_days

In [ ]:
date = datetime.now()
date_str = date.strftime( '%Y/%m/%d' ) 

(key, value), = bull_bear_days['bull_bear'][-1].items()
days = trading_days(value, date_str)
bull_days = bull_bear_days['bull_days']
bear_days = bull_bear_days['bear_days']
if key == 'bull': bull_days +=  days
if key == 'bear': bear_days +=  days


In [ ]:
all_days = 243

bull_days_str = str(bull_days) + ' days'
bear_days_str = str(bear_days) + ' days'
other_days = all_days - bull_days - bear_days
other_days_str = str(other_days) + ' days'
#bull_bear_days['bull_bear'].append({'bear': date_str})
print(bull_days_str,bear_days_str,other_days_str)